
Inspired by Quix's [simple-kafka-python](https://github.com/quixio/simple-kafka-python/tree/main).

# Main functions below, above is testing it coming together

## API docs
https://content.tfl.gov.uk/trackernet-data-services-guide-beta.pdf older one I think  
https://api-portal.tfl.gov.uk/api-details#api=Line&operation=Line_ArrivalsByPathIds&definition=Tfl-41 from portal

In [ ]:
import requests
import time
import json


def get_stop_point_info(stop_point_name,mode="tube"):
    if mode not in ["tube", "bus"]:
        raise ValueError("Invalid mode. Mode must be 'tube' or 'bus'.")
    
    response = requests.get(
        "https://api.tfl.gov.uk/StopPoint/Search/" + stop_point_name,
        params = {
            'app_key': app_key
        }
    )
    response_data = response.json()


    results = []


    matches = response_data['matches'] # 'matches' is part of the response

    for match in matches:
        if mode in match['modes']:
            results.append({
                'name': match['name'],
                'id': match['id'],
            })
    return results


def get_tube_arrivals(line_id, stop_point_id, direction, app_key):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
    response = requests.get(
        url,
        params = {
            'direction': direction,
            'app_key': app_key,
        }
    )
    return response.json()

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv() #load from your .env file
app_key = os.getenv('TFL_API_KEY')

line_id = "victoria"
stop_point_id = [id['id'] for id in get_stop_point_info("Brixton")] # gets the id out of the {name, id] result}
direction = "outbound"

In [ ]:
# Testing block

response = get_tube_arrivals(line_id, stop_point_id[0], direction, app_key)
print(response)

In [ ]:
import logging
from quixstreams import Application

def main():
    app = Application(
        broker_address="localhost:19092",
        loglevel="DEBUG",
    )

    with app.get_producer() as producer:
        while True:
            weather = get_weather()
            logging.debug("Got weather: %s", weather)
            producer.produce(
                topic="weather_data_demo",
                key="London",
                value=json.dumps(weather),
                headers= {"app.name": "python-quix"}
            )

            weather_moscow=get_weather_moscow()
            logging.debug("Got weather: %s", weather_moscow)
            producer.produce(
                topic="weather_data_demo",
                key = "Moscow",
                value=json.dumps(weather_moscow),
                headers= {"app.name": "python-quix"}
            )
            logging.info("Produced. Sleeping...")
            time.sleep(5)


if __name__ == "__main__":
    logging.basicConfig(level="DEBUG")
    main()